## Boilerplate

In [13]:
import plotly.graph_objects as go
import random
import numpy as np

In [2]:
def make_graph(x, y, yaxis_title="Counts", title=None, texttemplate='%{text}'):
    
    title = title or f"Sample histogram (Total Count = {sum(y):,})"
    
    fig = go.Figure(data=[go.Bar(
        x=x,
        y=y,
        text=y,
        textposition='inside',
        texttemplate=texttemplate,
        opacity=0.75,
        marker={
            'line':{
                'color': 'white',
                'width' : 1
            },
            'color' : '#b7410e'
        }
    )])

    fig.update_layout(
        bargap=0,
        xaxis=dict(
            tickmode='linear'
        ),
        xaxis_title="Sum of dice",

        yaxis=dict(
    #         showticklabels=False
            showgrid=False
        ),
        yaxis_title=yaxis_title,

        title=title,

    #     paper_bgcolor='#F5F5DC',
        plot_bgcolor='#F5F5DC',

        width=850,
        height=600,
        margin=dict(
            l=10,
            t=50,
            b=10
        )
    )

    return fig

Counts

In [3]:
def make_hist_dice(n, title=None):
    title = title or f"Simulation of {n:,} rolls"
    x = list(range(2, 13))
    y = 11*[0]
    for _ in range(n):
        d1 = random.randint(1, 6)
        d2 = random.randint(1, 6)
        sum_dice = d1 + d2
        y[sum_dice-2] += 1

    return make_graph(x, y, title=title)

Average

In [4]:
def make_hist_average_dice(n):
    x = list(range(2, 13))
    y = 11*[0]
    for _ in range(n):
        d1 = random.randint(1, 6)
        d2 = random.randint(1, 6)
        sum_dice = d1 + d2
        y[sum_dice-2] += 1
    
    y = [i/n for i in y]

    return make_graph(x, y, yaxis_title="Average value", title=f"Simulation of {n:,} rolls", texttemplate='%{text:.4f}')

Central Limit Theorem

In [5]:
def simulate_multiple_dice(num_dice, sample_size):
    x = list(range(2, 6*num_dice + 1))
    y = len(x)*[0]
    
    for _ in range(sample_size):
        dice_sum = 0
        for _ in range(num_dice):
            dice_sum += random.randint(1, 6)
        
        y[dice_sum-2] += 1
    
    return make_graph(x, y, yaxis_title="Count", title=f"Simulation of sum of {num_dice:,} dice over {sample_size:,} rolls", texttemplate='%{text}')

In [6]:
FORMATS = ['png', 'jpeg']

## Generating the actual plots

#### Sample / Dummy histogram

In [8]:
x = list(range(2, 13))
fake_counts = [1, 3, 0, 4, 4, 10, 8, 7, 3, 8, 2]
fig = make_graph(x, fake_counts)

for frmt in FORMATS:
    fig.write_image(f"Graphs/sample_histogram.{frmt}")

#### Simulation (count)

In [35]:
for total_rolls in [500, 5000, 50000, 500000]:
    fig = make_hist_dice(total_rolls)
    for frmt in FORMATS:
        fig.write_image(f"Graphs/histogram_dice_{total_rolls}.{frmt}")

#### Simulation (average)

In [20]:
for frmt in FORMATS:
    make_hist_average_dice(50000).write_image(f"Graphs/histogram_dice_average_{total_rolls}.{frmt}")

#### Visualizing bar chart of expected values

In [22]:
for frmt in FORMATS:
    make_graph(x=list(range(2,13)), y=[x/36 for x in [1,2,3,4,5,6,5,4,3,2,1]], yaxis_title="Value of E[X] vector", title="Visualization of E[X]").write_image(f"Graphs/histogram_E[X].{frmt}")

#### Central Limit Theorem

In [32]:
simulate_multiple_dice(5, 50000).write_image(f"Graphs/bell_curve_5dice_50000rolls.jpeg")

## Graph normal distribution with num of bins = 12

##### Generate 50000 Normal random values, mean 7 and some s.d.

In [62]:
y_values = np.random.normal(loc=7, scale=2.5, size=50000)
y_values_trimmed = [y for y in y_values if 2 <= y and y <= 12]

Want to go from 50000 y values to 12 buckets

\[5.6, 3.3, 6.7, 2.2, ..., 3.3, 7.7, 5.6\] <-- 50000 of these

\[234, 123, 2345, 12033, ..., 123\] <-- 12 buckets

##### Create range first:

In [63]:
focal_points = range(2, 13)
focal_ranges = []
for f in focal_points:
    focal_ranges.append((f - 0.5, f + 0.5))

##### Distribute the y values into buckets / focal_ranges

In [64]:
buckets = [0]*12
for y in y_values_trimmed:
    for i, f in enumerate(focal_ranges):
        lower_f, upper_f = f[0], f[1]
        if y >= lower_f and y < upper_f:
            buckets[i] += 1
            break

assert(sum(buckets) == len(y_values_trimmed))

x = list(range(2, 13))
make_graph(x, buckets).show()